In [0]:
!git clone https://github.com/dipsankarb/streaming.git

# Bloom Filter 

The Bloom Filtering is a technique that is popularly used for setting a filter on a stream so that only the items that have been already seen before. So it allows all stream elements whose keys are in a stream $S$ while rejecting most of the stream elements whose keys are not in $S$.

First define some helper methdos that can help in the implementation.

In [0]:
# We will use the bitarray library for defining the hash table
import numpy as np
!pip install bitarray
!pip install mmh3
from bitarray import bitarray

import random
import mmh3
import math


train_data_path = '/content/streaming/email/train.csv'

test_data_path = '/content/streaming/email/test.csv'
train_data = []
with open(train_data_path,'r') as f:
    data = f.read()
    data = data.split('\n')
    for d in data:
        train_data.append(d)

train_data = train_data[:-1]
# print(train_data)

# print(len(train_data))
test_data = []
with open(test_data_path,'r') as f:
    data = f.read()
    data = data.split('\n')
    for d in data:
        test_data.append(d)

test_data = test_data[:-1]
# print(test_data)

    

In [26]:
# ca and cb are hash function co-efficients
# val is the value to be hashed
# p is the current size of the hash table (the mod part, find a large prime number for this that is >p)

mod = 2**32 - 1
def getSize(n,prob):
    m = -(n * math.log(prob))/(math.log(2)**2) 
    return int(m)

def num_hashes(m,n):
    k = (m/n)*math.log(2)
    return int(k)

def generate_coef(k):
    i = 0
    l = []
    while i<k:
        x = random.randint(0,mod)
        while x in l:
            x = random.randint(0,mod)
            
        l.append(x)
        i += 1
        
    return l

n = len(train_data)

false_prob = 0.02

size_bitArray = getSize(n,false_prob)
k = num_hashes(size_bitArray,n)

coefficient_A = generate_coef(k)
coefficient_B = generate_coef(k)

print('BitArray size : ',size_bitArray,'\nNumber of hash functions : ',k)


bitArray = bitarray(size_bitArray)
bitArray.setall(0)


def add(val):
    for i in range(k):
        # x = mmh3.hash(val,i)%size_bitArray
        x = (coefficient_A[i]*encode(val) + coefficient_B[i])%size_bitArray
        updateFilter(x)

def check(val):
    for i in range(k):
        # x = mmh3.hash(val,i)%size_bitArray
        x = (coefficient_A[i]*encode(val) + coefficient_B[i])%size_bitArray

        if bitArray[x] == 0:
            return False
    return True
    

BitArray size :  16284 
Number of hash functions :  5


In [0]:
# Now set to 1 in the bitarray b from hashFunc()

def updateFilter(pos) : bitArray[pos] = 1

You all are aware of the probability relationship in Bloom Filters which can be given by:

$P = (1 - (1-\frac{1}{n})^{km})^k$

where n is the size of the bit-array and there are m items to be filtered.

In the following function set the size of the hash table (prime) which can be found out by using the formula derived from the above one as:

$n = -\frac{m\ln P}{(ln 2)^2}$

Additionally the number of hash functions can be computed as :

$k = \frac{n}{m} ln2$

Now input the data list (of emails from /email directory)

For each of the emails, find the unicode sum for all of it's characters using ord() function

In [0]:
def encode(val):
    en = sum(ord(c) for c in val)
    return en

In [0]:
# now generate k hash functions using random integers for ca and cb and maximum size p
for word in train_data:
    add(word)

    


Use the unicode sums as the val in hashFunc() to hash each email

Now input the test data and for every data, write the following check() function where it will go over the k hash functions and see if the email has been seen or not. If any of the bit is the bitarray is false, then the email does not exist else there is a probability P that it exists.

For the test data, show what is the value of P, what is size of bit-array, and how many are definietly not present, how many are probably present, and how many false positives are there in the bit array.

In [30]:
test_data.append('jhaabhinav1998@gmail.com')
test_data.append('abhijha07@yahoo.in')
# print(getSize(len(test_data),false_prob))
n_false,n_present,n_not_present = 0,0,0
for word in test_data:
    if check(word):
        if word not in train_data:
            n_false += 1
            # print("'{}' is a false positive!".format(word))
        else:
            n_present += 1
            # print("'{}' is probably present!".format(word))
    else:
        n_not_present += 1
        # print("'{}' is definitely not present!".format(word))

print('Occupancy rate of bitArray : ',(sum(bitArray)/len(bitArray)) * 100, '%')
print('No. of false positive : ',n_false)
print('No. of definitely not present : ',n_not_present)
print('No. of probably present : ',n_present)


Occupancy rate of bitArray :  25.878162613608453 %
No. of false positive :  1
No. of definitely not present :  1
No. of probably present :  500
